# Models Evaluation Notebook



Mount Google Drive to access your data



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install necessary notebooks


In [ ]:
%%capture
!pip install -e git+https://github.com/ncullen93/torchsample.git#egg=torchsample
!pip install visdom
!pip install nibabel
!pip install h5py
!pip install torchsample
!pip install tensorboardX

Import libraries

In [ ]:
#import all libraries
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision import models
import numpy as np
import os
import sys
import pickle
import torch.nn.functional as F
import torch.utils.data as data
import pandas as pd
from torch.autograd import Variable
# from src.torchsample.torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
# from torchvision import transforms
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter
import math
from sklearn import metrics
from torchsummary import summary
from torch.utils.data import DataLoader, random_split
from datetime import datetime

# Set a random seed for reproducibility
torch.manual_seed(42)

Assign Device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


Create Dataloader




In [ ]:
class Dataset(data.Dataset):
    def __init__(self, root_dir, task, plane, split='train', transform=None):
        super().__init__()
        self.task = task
        self.plane = plane
        self.root_dir = root_dir
        self.split=split
        if self.split == 'train':
            self.folder_path = self.root_dir + 'train/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'train-{0}.csv'.format(task), header=None, names=['id', 'label'])
        elif self.split == 'test':
            self.folder_path = self.root_dir + 'test/{0}/'.format(plane)
            self.records = pd.read_csv(
                self.root_dir + 'test-{0}.csv'.format(task), header=None, names=['id', 'label'])
        else:
            self.folder_path = self.root_dir + 'valid/{0}/'.format(plane)

            self.records = pd.read_csv(
                self.root_dir + 'valid-{0}.csv'.format(task), header=None, names=['id', 'label'])

        self.records['id'] = self.records['id'].map(
            lambda i: '0' * (4 - len(str(i))) + str(i))
        self.paths = [self.folder_path + filename +
                      '.npy' for filename in self.records['id'].tolist()]
        self.labels = self.records['label'].tolist()

        self.transform = transform

        pos = np.sum(self.labels)
        neg = len(self.labels) - pos
        self.weights = [1, neg / pos]


    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        array = np.load(self.paths[index]) #load MRI
        label = self.labels[index] #get label of MRI
        label = torch.FloatTensor([label]) #convert type from numpy to torch

        if self.transform: #if you are transforming it
            array = self.transform(array) #transform the image
            array = array.numpy()


        array = np.stack((array,)*3, axis=1) #the model expects dimensions of (3, 256, 256), the MRIs are greyscale of size (256, 256). Therefore, we stack the image three times to fit the dimensions for the model.
        array = torch.FloatTensor(array)

        if label.item() == 1:
            weight = np.array([self.weights[1]])
            weight = torch.FloatTensor(weight)
        else:
            weight = np.array([self.weights[0]])
            weight = torch.FloatTensor(weight)

        return array, label, weight

## Define models to be evaluated

In [ ]:
#add another fully connected layer to convert output (1,1000) to (1)
class BaselineNetAdapt(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model = nn.Sequential(*list(models.resnet18(weights='ResNet18_Weights.DEFAULT').children())[:-2])
        self.conv1 = nn.Conv2d(512,64,4)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64,32,4)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifer = nn.Linear(32,1)


    def forward(self, x):
        # input size of x (1, s, 3, 256, 256) where s is the number of slices in one MRI
        x = torch.squeeze(x, dim=0) #output size (s, 3, 256, 256)
        x = self.pretrained_model(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.avgpool(x)
        x = self.flatten(x)

        output = torch.max(x, 0, keepdim=True)[0] #output size (1, 1000)
        output =nn.ReLU()(output)
        output = self.classifer(output) #output size (1)

        return output

In [ ]:
ResNet18Adapt_model = BaselineNetAdapt().to(device)
ResNet18Adapt_model.name = 'ResNet18Adapt' # For evaluation
summary(ResNet18Adapt_model, (3, 256, 256))

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 61.8MB/s]


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           9,408
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5           [-1, 64, 64, 64]          36,864
       BatchNorm2d-6           [-1, 64, 64, 64]             128
              ReLU-7           [-1, 64, 64, 64]               0
            Conv2d-8           [-1, 64, 64, 64]          36,864
       BatchNorm2d-9           [-1, 64, 64, 64]             128
             ReLU-10           [-1, 64, 64, 64]               0
       BasicBlock-11           [-1, 64, 64, 64]               0
           Conv2d-12           [-1, 64, 64, 64]          36,864
      BatchNorm2d-13           [-1, 64, 64, 64]             128
             ReLU-14           [-1, 64,

In [ ]:
class AlexNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.pretrained_model = models.alexnet(pretrained=False)
        self.pooling_layer = nn.AdaptiveAvgPool2d(1)
        self.classifer = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.squeeze(x, dim=0)
        features = self.pretrained_model.features(x)
        pooled_features = self.pooling_layer(features)
        pooled_features = pooled_features.view(pooled_features.size(0), -1)
        flattened_features = torch.max(pooled_features, 0, keepdim=True)[0]
        output = self.classifer(flattened_features)
        return output

In [ ]:
AlexNet_model = AlexNet().to(device)
AlexNet_model.name = 'alexnet' # For evaluation
summary(AlexNet_model, (3, 256, 256))

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 63, 63]          23,296
              ReLU-2           [-1, 64, 63, 63]               0
         MaxPool2d-3           [-1, 64, 31, 31]               0
            Conv2d-4          [-1, 192, 31, 31]         307,392
              ReLU-5          [-1, 192, 31, 31]               0
         MaxPool2d-6          [-1, 192, 15, 15]               0
            Conv2d-7          [-1, 384, 15, 15]         663,936
              ReLU-8          [-1, 384, 15, 15]               0
            Conv2d-9          [-1, 256, 15, 15]         884,992
             ReLU-10          [-1, 256, 15, 15]               0
           Conv2d-11          [-1, 256, 15, 15]         590,080
             ReLU-12          [-1, 256, 15, 15]               0
        MaxPool2d-13            [-1, 256, 7, 7]               0
AdaptiveAvgPool2d-14            [-1, 25

Define Evaluation function

In [ ]:
def test(model,test_loader):
    _ = model.eval()
    y_trues = []
    aucs = []
    accuracys = []
    y_preds = []
    y_ps = []
    running_accuracy = 0
    total = 0

    with torch.no_grad():
        for i, (image, label, weight) in enumerate(test_loader):
          #print("i = ",i)
          if torch.cuda.is_available():
            image = image.cuda()
            label = label.cuda()
          label = label[0]
          #prediction = model.forward(image.float()).squeeze(0)
          prediction = model(image.float().squeeze(0))
          #print(label,prediction)

          probas = torch.sigmoid(prediction)
          #print('probas:', probas.item())

          preds = (prediction>0.5).float()
          preds_arr = (preds.detach().cpu().numpy()).flatten()
          y_ps.append(preds_arr)
          y_trues.append(int(label[0]))
          y_preds.append(probas[0].item())

          #print("preds ",preds)
          #print("preds_arr ",preds_arr)
          #print("y_ps ",y_ps)
          #print("y preds ",y_preds)
          #print("y trues ",y_trues)

          try:
            #auc = metrics.roc_auc_score(int(label[0]), probas.item().to_array())
            auc = metrics.roc_auc_score(y_trues, y_preds)
            accuracy = metrics.accuracy_score(y_trues, y_ps)
            #roc = metrics.roc_curve(y_trues, y_ps) # nice to have ??????????????????????????????????????
          except Exception as e:
            #print('Exception:', e)
            auc = 0.5
            accuracy = 0.5
            #roc = 0.5
          aucs.append(auc)
          #print("roc ",roc)
          accuracys.append(accuracy)
          writer.add_scalar('Test/AUC', auc, i)
          writer.add_scalar('Test/Accuracy', accuracy, i)
          #writer.add_scalar('Test/ROC', roc, i)
    auc_average = sum(aucs)/len(aucs)
    acc_average = sum(accuracys)/len(accuracys)

    return auc_average, acc_average

Activate TensorBoard

In [ ]:
%load_ext tensorboard

Define variables to summarize the evaluation results

In [ ]:
history_auc = {'res_net_axial': 0,'res_net_coronal': 0,'res_net_sagital': 0,'Alex_net_axial': 0,'Alex_net_coronal': 0,'Alex_net_sagital': 0}

In [ ]:
history_acc = {'res_net_axial': 0,'res_net_coronal': 0,'res_net_sagital': 0,'Alex_net_axial': 0,'Alex_net_coronal': 0,'Alex_net_sagital': 0}

## Evaluate models

### ResNet 18 Adapt


Evaluation for ACL tear and Axial plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'axial' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
ResNet18Adapt_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/ResNet18Adapt_model_weights_axial.pth"))


<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(ResNet18Adapt_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['res_net_axial'] = auc_average
print(history_auc)
history_acc['res_net_axial'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0, 'res_net_sagital': 0, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0, 'res_net_sagital': 0, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}


Evaluation for ACL tear and Cornal plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'coronal' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
ResNet18Adapt_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/ResNet18Adapt_model_weights_coronal.pth"))

<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(ResNet18Adapt_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['res_net_coronal'] = auc_average
print(history_auc)
history_acc['res_net_coronal'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0.5, 'res_net_sagital': 0, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0.8544769850299887, 'res_net_sagital': 0, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}


Evaluation for ACL tear and Sagittal plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'sagittal' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
ResNet18Adapt_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/ResNet18Adapt_model_weights_sagittal.pth"))

<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(ResNet18Adapt_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['res_net_sagital'] = auc_average
print(history_auc)
history_acc['res_net_sagital'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0.5, 'res_net_sagital': 0.9212772947592173, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0.8544769850299887, 'res_net_sagital': 0.8969074097351194, 'Alex_net_axial': 0, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}


### AlexNet

Evaluation of model with ACL tear and Axial plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'axial' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
AlexNet_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/AlexNet_model_Axial_ACL_weights.pth"))

<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(AlexNet_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['Alex_net_axial'] = auc_average
print(history_auc)
history_acc['Alex_net_axial'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0.5, 'res_net_sagital': 0.9212772947592173, 'Alex_net_axial': 0.5, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0.8544769850299887, 'res_net_sagital': 0.8969074097351194, 'Alex_net_axial': 0.857363525712419, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0}


Evaluation of model with ACL tear and Sagittal plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'sagittal' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
AlexNet_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/AlexNet_model_weights_sagittal.pth"))

<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(AlexNet_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['Alex_net_sagital'] = auc_average
print(history_auc)
history_acc['Alex_net_sagital'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0.5, 'res_net_sagital': 0.9212772947592173, 'Alex_net_axial': 0.5, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0.5}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0.8544769850299887, 'res_net_sagital': 0.8969074097351194, 'Alex_net_axial': 0.857363525712419, 'Alex_net_coronal': 0, 'Alex_net_sagital': 0.8516101908502639}


Evaluation of model with ACL tear and Coronal plane

In [ ]:
directory = '/content/gdrive/Shareddrives/MRNet Project/MRNet-v1.0/MRNet-v1.0/'
task = 'acl'
plane = 'coronal' # Coronal #
# initialise the train and validation datasets (class we defined earlier) and then initialise a Pytorch's dataloader
test_dataset = Dataset(directory, task, plane, split='test', transform = None)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=-True, num_workers=2, drop_last=False)

In [ ]:
AlexNet_model.load_state_dict(torch.load("/content/gdrive/Shareddrives/MRNet Project/model_weights/AlexNet_model_weights_coronal.pth"))

<All keys matched successfully>

In [ ]:
# - Evauation Start -
log_root_folder = "./gdrive/MyDrive/assignment/logs/{0}/{1}/".format(task, plane)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)
# - Evaluation End -
auc_average, acc_average = test(AlexNet_model,test_loader) # ResNet18Adapt_model, AlexNet_model
history_auc['Alex_net_coronal'] = auc_average
print(history_auc)
history_acc['Alex_net_coronal'] = acc_average
print(history_acc)
writer.close()

{'res_net_axial': 0.6589337060278105, 'res_net_coronal': 0.5, 'res_net_sagital': 0.9212772947592173, 'Alex_net_axial': 0.5, 'Alex_net_coronal': 0.5, 'Alex_net_sagital': 0.5}
{'res_net_axial': 0.7838804727459848, 'res_net_coronal': 0.8544769850299887, 'res_net_sagital': 0.8969074097351194, 'Alex_net_axial': 0.857363525712419, 'Alex_net_coronal': 0.8748395876020522, 'Alex_net_sagital': 0.8516101908502639}


Insert this snippet into browser developer tools console to ensure that colab runtime does not disconnect.

`Note: This does not go against the ToS of Google Colab`

```python
function ClickConnect(){
console.log("Working");
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click()
}
setInterval(ClickConnect,60000)
```